In [28]:
# Paso 0 - se procede a importar todas las librerias necesarias 
import pydicom
from pydicom import dcmread 
import os
import random
import dicom2nifti
import nilearn
from nilearn import plotting

In [48]:
# Paso 3 - Explicacion de metodos y atributos de la clase 'Estudio'
class Estudio: 

    def __init__(self,ruta_carpetaEstudio, path_carpetaTrabajo): #Se inicializan los atributos de clase
        self.ruta_estudio = ruta_carpetaEstudio
        self._imgCarpeta = os.listdir(rf'{self.ruta_estudio}')
        self.path_carpetaTrabajo = path_carpetaTrabajo
         
        Paciente.agregar_estudioPaciente(self)
        Paciente.añadirDatos_list(self)                 #Se realiza la relacion de dependencia con la clase Paciente y SistemaGestion

    def verDiccionario(self):
        Paciente.mostrarDic(self)

    def informacionEstudio(self):
        SistemaGestion._manejoEstudios(self)     #Se crear metodos para llamar a otros metodos de otras clases, para que asi solo se tenga que crear un objeto

       
    def _anonimizar(self):  

        nombreEstudio_anonimus = input("Ingrese un nombre o codigo para el estudio: ")   

        lista_nombreCarpeta1 = os.listdir(rf"{self.path_carpetaTrabajo}") #Se crea un apartado para crear una carpeta y evitar errores, y se              
                                                                          # procesan todos los paths necesarios  
        nuevaCarpeta_anonimus = os.path.join(self.path_carpetaTrabajo,f'CarpetaDatos_Anonimizados_{nombreEstudio_anonimus}')

        ruta_nuevaCarpeta_anonimus = os.path.abspath(f'CarpetaDatos_Anonimizados_{nombreEstudio_anonimus}')

        if f'CarpetaDatos_Anonimizados_{nombreEstudio_anonimus}' in lista_nombreCarpeta1:

            pass  #Se genera la condicion para evitar errores
        
        else:
            os.mkdir(nuevaCarpeta_anonimus)


        id = f'P{random.randint(1,4560)}I{random.randint(1,4560)}D'
        name = f'P{random.randint(1,4560)}N'
        sex = f'P{random.randint(1,4560)}S'  #Se generan los valores randoms para anonimizar los datos
        
        for i in range(len(self._imgCarpeta)):
        
            archivo = pydicom.dcmread(os.path.join(self.ruta_estudio,self._imgCarpeta[i]))
            archivo['PatientID'].value = id
            archivo['PatientBirthDate'].value= None
            archivo['PatientName'].value = name   #Se anonimizan todos los datos por medio de un for para ahorrar codigo
            archivo['PatientSex'].value= sex
            archivo.save_as(f'{os.path.join(ruta_nuevaCarpeta_anonimus,self._imgCarpeta[i])}')

    def convertirEstudio(self):

        nombreEstudio = input("Ingrese un nombre o codigo para el estudio: ") #Se crea un apartado para crear una carpeta y evitar errores, y se              
                                                                              # procesan todos los paths necesarios   
        lista_nombreCarpeta = os.listdir(rf"{self.path_carpetaTrabajo}")                
        
        nuevaCarpeta = os.path.join(self.path_carpetaTrabajo,f'CarpetaNifti_{nombreEstudio}')

        if f'CarpetaNifti_{nombreEstudio}' in lista_nombreCarpeta:

            pass
        
        else:
            os.mkdir(nuevaCarpeta)

        try:     
    
            dicom2nifti.convert_directory(self.ruta_estudio,nuevaCarpeta)
            print('Carpeta creada exitosamente')  #Se convierten los datos de DICOM a NIFTI

        except :
            print('Ha ocurrdio un error')


    def visualizarNilearn(self,ruta_ArchivoNifti):

            try:
                imagen = nilearn.image.load_img(rf'{ruta_ArchivoNifti}')  #Se crea un menu para seleccionar modo de vista

                menu_ppl = int(input("""
                                                Seleccione el modo de vista deseado:

                                                        1) Un solo plano
                                    
                                                        2) Los tres planos 
                                    
                                                        3) Mosaico
                                    
                                                        4) Salir

                                                                            Respuesta: """))
                if menu_ppl not in range(1,5):
                    raise ValueError
            
                

                if menu_ppl == 1:

                        try:  #Se genera un submenu para escoger el plano deseado
                            menuSec = int(input("""
                                                    Seleccione el plano que desea ver:
                                                
                                                        1) Axial
                                                
                                                        2) Coronal
                                                
                                                        3) Sagital
                                                
                                                        4) Salir
                                                    
                                                                        Respuesta:  """))
                            
                            if menuSec not in range(1,5):
                                raise ValueError #Se usa el raise para que el usuario no pueda ingresar una opcion distinta a las ofrecidas
                            
                            if menuSec == 1:
                                
                                SistemaGestion.infoImagen(self,ruta_ArchivoNifti) #Se llama al metodo de SistemaGestion para que se muestre la info necesaria
                                plotting.plot_anat(imagen, display_mode='z', title='Plano axial') #Se plotea la imagen con este metodo

                            if menuSec == 2:

                                SistemaGestion.infoImagen(self, ruta_ArchivoNifti)
                                plotting.plot_anat(imagen, display_mode='y', title='Plano coronal')

                            if menuSec == 3: 
                                
                                SistemaGestion.infoImagen(self, ruta_ArchivoNifti)
                                plotting.plot_anat(imagen, display_mode='x', title='Plano sagital')

                            if menuSec == 4:

                                pass
                        except:
                            print("No se aceptan letras y solo se aceptan numeros que esten en el rango de 1 a 4")     
                        #Se usa un Try-Except para evitar errores de input y conversion a int
                if menu_ppl == 2:
                    
                    SistemaGestion.infoImagen(self, ruta_ArchivoNifti)
                    plotting.plot_anat(imagen, display_mode='ortho', title='Planos Coronal, Sagital y Axial')

                if menu_ppl == 3:
                    
                    SistemaGestion.infoImagen(self, ruta_ArchivoNifti)
                    plotting.plot_anat(imagen,title='Mosaico',display_mode="mosaic")

                if menu_ppl == 4:

                    pass
            except :
                print("No se aceptan letras y solo se aceptan numeros que esten en el rango de 1 a 4")

# Paso 4 - Explicacion de metodos y atributos de la clase 'Paciente'
class Paciente(Estudio):

    estudiosPaciente = {}  #Se crea un diccionario donde se guardara toda el Id de los pacientes con sus respectivos estudios.

    def __init__(self, ruta_carpetaEstudio, path_carpetaTrabajo):
        super().__init__(ruta_carpetaEstudio, path_carpetaTrabajo)

    

    def _obtencionID(self): #Se crea la funcion para obtener el Id del archivo DICOM
        archivo = pydicom.dcmread(os.path.join(self.ruta_estudio,self._imgCarpeta[0]))
        archivo = archivo.PatientID
        return archivo

    def añadirDatos_list(self):  #Se me añada los id obtenidos a una lista que se encuentra en la clase 'SistemaGestion'
        iD1 = Paciente._obtencionID(self)
        if iD1 in SistemaGestion.id_list:
            pass
        else:
            SistemaGestion.id_list.append(iD1)  

    def agregar_estudioPaciente(self): 
        iD = Paciente._obtencionID(self)
        llaves = Paciente.estudiosPaciente.keys()#Por medio de condicionales y metodos de diccionario se filtra, agrega y valida, la entrada de nuevos
                                                 #pacientes y estudios relacionados a estos mismos  

        if iD in llaves:
            comprobante = Paciente.estudiosPaciente[iD]
            if self.ruta_estudio in comprobante:
                pass
            else:
                Paciente.estudiosPaciente[iD] += [self.ruta_estudio]     

        else:
            Paciente.estudiosPaciente[iD] = [self.ruta_estudio]

    def mostrarDic(self):
        print(Paciente.estudiosPaciente) #Se crea este metodo para visualizar la entrada y modificacion de los datos en el codigo

# Paso 5 - Explicacion de metodos y atributos de la clase 'SistemaGestion'
class SistemaGestion (Estudio): # Se crea la clase SistemaGestion, inicializando los atributos de la clase padre (Estudio) y añadiendo un nuevo atributo de clase

    id_list = []    
        
    def __init__(self, ruta_carpetaEstudio, path_carpetaTrabajo):
        super().__init__(ruta_carpetaEstudio, path_carpetaTrabajo)
       

    def _manejoEstudios (self): # Se define método que contiene un menú con opciones para el manejo de los estudios
        
        try: 
            menu_est = int(input('''Ingrese el número de la acción que desea realizar:
                            1. Añadir estudio.
                            2. Consultar información de un estudio.
                            3. Eliminar estudio.
                            4. Salir'''))
            
            if menu_est not in range(1,5):
                raise ValueError
        
            if menu_est == 1: # En esta opción del menú se permite agregar un estudio, reutilizando parte del código de obtencionID de la clase Paciente y utilizando atributos de la misma.

                ruta_estudio = input('Ingrese la ruta de la carpeta del estudio que quiere añadir: \n')
                ruta_estudio = rf'{ruta_estudio}'
                iD = pydicom.dcmread(os.path.join(ruta_estudio,self._imgCarpeta[0]))
                iD = iD.PatientID
                if iD in SistemaGestion.id_list:
                    comprobante = Paciente.estudiosPaciente[iD]#Además, como se hizo en una parte anterior a esta, se realiza comprobación para que el estudio no sea añadido dos veces
                    if ruta_estudio in comprobante:
                        print(f'El estudio ya esta registrado en el paciente con id: {iD} ')
                        pass
                    else:
                        Paciente.estudiosPaciente[iD] += [ruta_estudio]     
                        print(f'El estudio se agregó al paciente con id: {iD} correctamente')
                else:
                    Paciente.estudiosPaciente[iD] = [ruta_estudio]
                    print(f'Se creo un nuevo paciente con id: {iD} y se le agregro el estudio {ruta_estudio} correctamente')

            elif menu_est == 2: #En esta opción del menú se enlistan los pacientes ya guardados para dar la opción de elegir el paciente que se quiere consultar
                print('Los pacientes con estudios registrados son los siguientes (ID):')
                for i in SistemaGestion.id_list:
                    print(f'{SistemaGestion.id_list.index(i) + 1} : {i}\n')
                
                try:   
                    elección = int (input('Ingrese el índice del ID del paciente sobre el desea consultar los estudios: '))
                    elección -= 1
                    paciente = SistemaGestion.id_list[elección]
                    print (f'El(los) estudio(s) registrado(s) del paciente con id {paciente} son: {Paciente.estudiosPaciente[paciente]}')#Después de la elección se enlista 
                    est = input('Ingrese el numero del estudio que quiere consultar: ')#nuevamente los estudios del paciente seleccionado para dar la opción de elegir sobre que estudio se quiere mostrar la información
                    imgCarpeta = os.listdir(rf'{est}')
                    Estudio = pydicom.dcmread(os.path.join(est,imgCarpeta[0]))
#Se muestra la información tanto del paciente como el estudio seleccionado, filtrando en cada caso el header del estudio para obtener la información que queremos
                    infoPaciente = Estudio.dir('Patient')
                    print('\nINFORMACIÓN DEL PACIENTE')
                    for i in infoPaciente:
                        print (f'--> {i} : {Estudio[i].value}')

                    infoEstudio = Estudio.dir('Study')   
                    print('\nINFORMACIÓN DEL ESTUDIO')
                    for s in infoEstudio:
                        print (f'--> {s} : {Estudio[s].value}')

                except: 
                    print("No se aceptan letras y solo se aceptan los numeros que se presentan en pantalla")
#Como ultima opción, se presenta de nuevo los pacientes y los estudios registrados para dar la opción de elegir que estudio se quiere eliminar
            elif menu_est == 3:
                print('Los pacientes con estudios registrados son los siguientes (ID):')
                for i in SistemaGestion.id_list:
                    print(f'{SistemaGestion.id_list.index(i) + 1} : {i}\n')
                
                try:   
                    elección = int (input('Ingrese el índice del ID del paciente sobre el desea consultar los estudios: '))
                    elección -= 1
                    paciente = SistemaGestion.id_list[elección]
                    print (f'El(los) estudio(s) registrado(s) del paciente con id {paciente} son: {Paciente.estudiosPaciente[paciente]}')
                    est = input('Ingrese el numero del estudio que quiere eliminar: ')
                    if est in Paciente.estudiosPaciente[paciente]:
                        Paciente.estudiosPaciente[paciente].remove(est)#Se hace uso de nuevo del diccionario creado en la clase Paciente para eliminar el estudio que el usuario ingrese mediante el metodo remove.
                        print('El estudio se ha removido exitosamente')
               
                except: 
                    print("No se aceptan letras y solo se aceptan los numeros que se presentan en pantalla")
                    
            elif menu_est == 4:
                pass

        except: 
            print("No se aceptan letras y solo se aceptan los numeros que se presentan en pantalla")
    
    def infoImagen(self, path_nifti):
        img= rf'{path_nifti}'
        img1=nilearn.image.load_img(img) 
        shape = img1.shape
        print('\n INFORMACiÓN DE LA IMAGEN')
        print(f"--> Pixeles en el ancho: {shape[0]}")
        print(f"--> Pixeles en la altura: {shape[1]}")
        print(f"--> Cortes: {shape[2]}")

# Se crea un método para mostrar la información de una imagen proyectada usando nilearn 
#(método que se llama en la parte del plotting de los estudios nifti) y extrae información sobre la forma de la misma.
        

In [64]:
#Paso 6 - Explicacion e implementacion de las pruebas

#prueba1 = Estudio(r'0001',r'C:\Documentos\Udea Clases\Tercer Semestre\Informatica 2\Trabajo_tercera_unidad')
#prueba2 = Estudio(r'0002',r'C:\Documentos\Udea Clases\Tercer Semestre\Informatica 2\Trabajo_tercera_unidad')
#prueba3 = Estudio(r'0003',r'C:\Documentos\Udea Clases\Tercer Semestre\Informatica 2\Trabajo_tercera_unidad')
#prueba1.informacionEstudio()
#prueba1.convertirEstudio()
#prueba1.visualizarNilearn(r'CarpetaNifti_0001\1_rm_cerebro_t1.nii.gz')

#prueba1.verDiccionario()


Los pacientes con estudios registrados son los siguientes (ID):
1 : IUIJNE

2 : RCXTEI

3 : XVKWSD

El(los) estudio(s) registrado(s) del paciente con id IUIJNE son: ['0001']

INFORMACIÓN DEL PACIENTE
--> ImageOrientationPatient : [-.0430156, .999005, .0117336, -.025947, .0106234, -.999607]
--> ImagePositionPatient : [108.954, -91.6572, 56.0041]
--> PatientBirthDate : 
--> PatientID : IUIJNE
--> PatientName : Anonymous
--> PatientPosition : HFS
--> PatientSex : F

INFORMACIÓN DEL ESTUDIO
--> StudyDate : 20230802
--> StudyDescription : CEREBRO SIMPLE
--> StudyID : 7255692790
--> StudyInstanceUID : 1.3.46.670589.11.45555.5.0.2240.2023080214011977002
--> StudyTime : 140119
{'IUIJNE': ['0001'], 'RCXTEI': ['0002'], 'XVKWSD': ['0003']}


In [59]:
#prueba1._anonimizar()

In [65]:
img_prueba = pydicom.dcmread(r"CarpetaDatos_Anonimizados_0001\IMG0001.dcm")
img_prueba.PatientName

'P1817N'